# Boy through time - video only

This notebook only performs the video generation part. For image alignment, see the `01_alignment` notebook.

# 1. Setup environment

Before we can start we need to make sure that all relevant packages are installed. We can do this using `pip`.

In [ ]:
!pip install -Uq  matplotlib~=3.8 numpy~=1.26 pathlib~=1.0 Pillow~=10.0 scikit-image~=0.22 tqdm~=4.66 pandas

Collect relevant folders and files from github repository. This is only needed when you don't run the notebook from within the repo, e.g. running it via colab.

In [ ]:
# Get github repository data - needed for when you run it via google's colab
!git clone https://github.com/miykael/boy_ages.git

## 1.1. Import relevant packages

In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage import io
from dateutil.relativedelta import relativedelta

from tqdm.notebook import tqdm
from PIL import Image, ImageDraw, ImageFont

## 1.2. Specify relevant parameters

In [ ]:
# Define size of face chip (maximum width or height of raw image)
max_size = 4320

# Downsample factor for face finding
downsample = 2

# Clean up temporary images needed for video generation
clean_up = True

## 1.3. Establish paths to images

The code in this notebook should run slightly different, depending on if you are running this notebook in google colab or not. So let's detect this:

In [ ]:
try:
    import google.colab

    IN_COLAB = True
except:
    IN_COLAB = False

In [ ]:
if IN_COLAB:
    # Get photo filenames from google drive
    from google.colab import drive

    gdrive = "/content/gdrive/"
    drive.mount(gdrive, force_remount=True)

    # Path to folder of original photos
    basename = Path(f"{gdrive}/MyDrive/photos_child")

else:
    # Path to local folder
    basename = Path("c:/Users/brend/desktop/cartertimelapse/photos_child_aligned/")

# 2. Identify filenames and dates of aligned images

Only select images in the `photos_child_aligned` foler where the file name starts with `PXL_` and ends with `x.jpg`.

In [ ]:
# Update the file_identifier
file_identifier = "PXL_*x.jpg"

In [ ]:
# Collect file names of aligned photos
imgs_aligned = sorted(basename.glob(file_identifier))
n_files = len(imgs_aligned)
print(f"{n_files} images were found, representing {n_files / 365.25:.2f} years.")

In [ ]:
# Collect dates from the imgs_aligned
dates = [f.name.split("PXL_")[1][:8] for f in imgs_aligned]

## 2.1. Create average image

In [ ]:
# Check if average file exists. Delete this image from the folder if you want it to run a new copy. Will skip if it already exists.
filename_avg = basename.parent / "img_01_avg.png"

In [ ]:
# Only run if 'img_01_avg.png' does not exist yet
if not filename_avg.exists():
    # Collect all images
    img_avg = np.zeros_like(io.imread(imgs_aligned[0])[..., :3], dtype="float32")
    for path_align in tqdm(imgs_aligned):
        img_avg += io.imread(path_align)[..., :3]

    # Scale data to be within 0 to 1 range
    img_avg = img_avg / len(imgs_aligned) / 255.0

    # Plot average images
    plt.figure(figsize=(7, 7))
    plt.imshow(img_avg)
    plt.axis("off")
    plt.show()

    # Store average images to file
    plt.imsave(filename_avg, img_avg)

# 3. Create videos

## 3.1. Video 1: No averaging

The first video is a simple stacking of all images and each image is given an index number. Note: The font for the text in the image was taken from [here](https://fonts.google.com/).

In [ ]:
# Specify frames per second
fps = 30

# Extract number of images
N_total = len(imgs_aligned)

print("Video length: %.2f seconds." % (N_total / fps))

In [ ]:
# Save temporary folder to store images to disk
tmp_dir = Path(basename / "tmp_video_imgs")
if not tmp_dir.exists():
    tmp_dir.mkdir(parents=True, exist_ok=True)

In [ ]:
# Define name of tmpd dir and filename of video 1
tmp_dir_name = str(tmp_dir)
video_filename = basename.parent / "video_01_aligned.mp4"
video_name = str(video_filename)

In [ ]:
# Collect images with text
filenames_img = []

# Loop through all images and add text
for idx in tqdm(np.arange(len(imgs_aligned))):
    # Establish out_filename
    out_filename_img = tmp_dir / f"{idx+1:05d}_image.jpg"
    out_filename_txt = tmp_dir / f"{idx+1:05d}_text.jpg"

    if not out_filename_txt.exists():
        # Load aligned image
        im = io.imread(imgs_aligned[idx])

        # Store image to file
        io.imsave(out_filename_img, np.array(im))

        # Establish date string
        date_str = dates[idx]

        # Compute delta in days to fixed start date
        day_delta = relativedelta(pd.to_datetime(date_str, format="%Y%m%d"), pd.to_datetime(dates[0], format="%Y%m%d"))

        # Collect exact year, month and day offset
        n_year, n_month, n_days = day_delta.years, day_delta.months, day_delta.days

        # Establish text on image
        title_txt = f"{date_str[4:6]} / {date_str[6:]} / {date_str[:4]} - Day #{idx+1:05d}\n"
        title_txt += f"{n_year:02d} years, {n_month:02d} months, {n_days+1:02d} days"

        # Add text to output filename
        img_txt = Image.fromarray(im)
        draw = ImageDraw.Draw(img_txt)
        width = max_size // downsample
        font_path = ""
        if IN_COLAB:
            font_path = "boy_ages/"

        # Set font, font size and x/y location of top left corner
        font_size = 0.06
        xx, yy = width * 0.13, width * 0.86

        # Draw background for text field
        text_size = (1650, 250)
        padding = 10  # Padding around text
        draw.rectangle(
            [xx - padding, yy - padding, xx + text_size[0] + padding, yy + text_size[1] + padding * 4],
            fill=(16, 16, 16, 128),
        )

        # Write text to image
        font = ImageFont.truetype(f"{font_path}Roboto/Roboto-Light.ttf", int(width * font_size))
        draw.text((xx, yy), title_txt, (255, 255, 255), font=font)

        # Store image with text to file
        io.imsave(out_filename_txt, np.array(img_txt))

    # Add image with text to list
    filenames_img.append(out_filename_img)

In [ ]:
# Skips creating a video if video already exists. Delete the file if you want an updated one created.
if not video_filename.exists():
    # Takes all images from tmp-folder and creates a video out of it with ffmpeg
    !ffmpeg -y -framerate $fps -i "$tmp_dir_name\%05d_text.jpg" -vcodec mjpeg -vcodec libx264 $video_name
    #!cat $tmp_dir_name/*_text.jpg | ffmpeg -y -f image2pipe -r $fps -vcodec mjpeg -i - -vcodec libx264 $video_name

# 3.2. Video 2: Smoothed / averaged images

The second video takes multiple images at once, averages them and then stacks them, leading to a temporal smoothing effect. Note: The font for the text in the image was taken from [here](https://fonts.google.com/).

In [ ]:
# Specify frames per second
fps = 30

In [ ]:
# Decide for mean or median averaging
use_mean = True
if use_mean:
    suffix = "mean"
else:
    suffix = "median"

### Adaptive smoothing and fps

The following parameter guides the adaptive smoothing / averaging and dynamic fps routine. Take for example:

```python
smooth_dict = {0: [7, 3],
               14: [14, 15],
               90: [30, 30],
               1095: [60, 30]}
```

This code means that from photo 0 (in Python that's the 1st one) to photo 14, the average is created over the last 7 days and this period uses an fps of 3 images/second (=3 days/second). Then from photo 14 to 90, images are averaged over last 14 days with an fps of 15. From photo 90 till 1095 (= 3 years) images are averaged over last 30 images with an fps of 30. And after that the average is done over the last 60 days, with an fps of 30.

Depending on how you set `smooth_dict` you can have different kinds of videos:

**Option 1**
```python
smooth_dict = {0: [30, 30]}
```
This will average each image over last 30 days and uses a constant fps of 30.

**Option 2**
```python
smooth_dict = {0: [7, 30],
               14: [14, 30],
               90: [30, 30],
               1095: [60, 30]}
```
This will have different averages. First 14 days only average over 7 days, then over 14 days, ...

**Option 3**
```python
smooth_dict = {0: [7, 3],
               14: [14, 15],
               90: [30, 30],
               1095: [60, 30]}
```
This changes days to average and fps.

**Option 4**
```python
smooth_dict = {0: [1, 1],
               30: [30, 30]}
```
This means that the first 30 days are not averaged at all and each image is shown for 1 second.

**Option 5**
```python
# Linearly increase the frame rate to 30 fps, for the first 50 images
linear_increase = np.linspace(1, 30, num=50, dtype='int')
smooth_dict = dict([[idx, [1, linear_increase[idx]]] for idx in np.arange(50)])

# At 30 fps, linearly increase the averaging of images from 1 to 30 for images 50 to 200
offset = 50
linear_increase = np.linspace(1, 30, num=150, dtype='int')
smooth_dict.update(dict([[idx + offset, [linear_increase[idx], 30]] for idx in np.arange(150)]))

# Keep constant average of 30 with 30 fps, from image 200 on
smooth_dict[200]= [30, 30]
```
This means that the first 50 days will not be averaged (hence the `1` in `[1, linear_increase[idx]]`), and will gradually speed up the frame rate to 30 fps, over the first 50 days. After that it will gradually speed up the number of images it afterages for 150 days (i.e. day 50 to 200). And lastly, from day 200 on, it will keep constant averaging and fps of 30.

In [ ]:
# Linearly increase the frame rate to 30 fps, for the first 50 images
linear_increase = np.linspace(1, 30, num=50, dtype="int")
smooth_dict = dict([[idx, [1, linear_increase[idx]]] for idx in np.arange(50)])

# At 30 fps, linearly increase the averaging of images from 1 to 30 for images 50 to 200
offset = 50
linear_increase = np.linspace(1, 30, num=150, dtype="int")
smooth_dict.update(dict([[idx + offset, [linear_increase[idx], 30]] for idx in np.arange(150)]))

# Keep constant average of 30 with 30 fps, from image 200 on
smooth_dict[200] = [30, 30]

In [ ]:
# Indicate if averaging should happen. False means it averages, True means no averaging
no_avg = False

In [ ]:
# Loop through images and average them according to smooth_dict
idx = 0
fdx = 0
smooth = 1

n_images = len(filenames_img)
pbar = tqdm()
while True:
    # Ignore loop if all images were processed
    if idx >= len(filenames_img) and np.std(imgs_idx) == 0:
        break

    # Acquire temporal smoothing length and fps duplication factor
    if idx in smooth_dict.keys():
        smooth, fps_fact = smooth_dict[idx]

    # Collect indeces of images
    imgs_idx = np.clip(np.arange(idx - smooth, idx) + 1, 0, n_images - 1)

    # Collect images
    imgs_group = np.array([io.imread(filenames_img[loc])[..., :3] for loc in imgs_idx])

    # Only keep one image if no averaging requested
    if no_avg:
        img_comp = imgs_group[-1].astype("uint8")
    else:
        # Average images
        if use_mean:
            img_comp = np.mean(imgs_group, axis=0).astype("uint8")
        else:
            img_comp = np.median(imgs_group, axis=0).astype("uint8")

    # Establish date string
    date_str = np.array(dates)[imgs_idx]
    date_str = date_str[-1]

    # Compute delta in days to fixed start date
    days_total = (pd.to_datetime(date_str, format="%Y%m%d") - pd.to_datetime(dates[0], format="%Y%m%d")).days
    day_delta = relativedelta(pd.to_datetime(date_str, format="%Y%m%d"), pd.to_datetime(dates[0], format="%Y%m%d"))

    # Collect exact year, month and day offset
    n_year, n_month, n_days = day_delta.years, day_delta.months, day_delta.days

    # Establish text on image
    title_txt = f"{date_str[4:6]} / {date_str[6:]} / {date_str[:4]} - Day #{days_total+1:05d}\n"
    title_txt += f"{n_year:02d} years, {n_month:02d} months, {n_days+1:02d} days"

    # Add text to output filename
    img_txt = Image.fromarray(img_comp)
    draw = ImageDraw.Draw(img_txt)
    width = max_size // downsample
    font_path = ""
    if IN_COLAB:
        font_path = "boy_ages/"

    # Set font, font size and x/y location of top left corner
    font_size = 0.06
    xx, yy = width * 0.13, width * 0.86

    # Draw background for text field
    text_size = (1650, 250)
    padding = 10  # Padding around text
    draw.rectangle(
        [xx - padding, yy - padding, xx + text_size[0] + padding, yy + text_size[1] + padding * 4],
        fill=(16, 16, 16, 128),
    )

    # Write text to image
    font = ImageFont.truetype(f"{font_path}Roboto/Roboto-Light.ttf", int(width * font_size))
    draw.text((xx, yy), title_txt, (255, 255, 255), font=font)

    # Create output file and duplicate it if necessairy (to decrease fps)
    duplication_factor = np.maximum(int(fps / fps_fact), 1)  # needs to be at least 1
    for _ in range(duplication_factor):
        # Save composition image
        out_filename = tmp_dir / f"{fdx+1:05d}_{suffix}.jpg"
        io.imsave(out_filename, np.array(img_txt).astype("uint8"))
        fdx += 1

    # Report progress
    if idx % 30 == 0:
        ddx = np.minimum(idx, len(dates) - 1)
        print(f"Finished date {dates[ddx]}")

    # Increase indecies
    idx += 1

    # Update the progress bar
    pbar.update(1)

# Close the progress bar
pbar.close()

In [ ]:
# Define name of tmpd dir and filename of video 2
tmp_dir_name = str(tmp_dir)
video_name = str(basename.parent / f"video_02_smoothed_{suffix}.mp4")

In [ ]:
# Takes all images from tmp-folder and creates a video out of it with ffmpeg
!ffmpeg -y -framerate $fps -i "$tmp_dir_name\%05d_{suffix}.jpg" -vcodec mjpeg -vcodec libx264 $video_name
#!cat $tmp_dir_name/*jpg | ffmpeg -y -f image2pipe -r $fps -vcodec mjpeg -i - -vcodec libx264 $video_name

## 3.3 Clean up

Remove all jpg files in the temporary directory that was used to create the frames for the video.

In [ ]:
# Remove all temporary files
if clean_up:
    for p in sorted(tmp_dir.glob("*jpg")):
        p.unlink()